<font size = 6>**Neon blink detection**

In [1]:
import pathlib
import numpy as np

from blink_detector.blink_detector import (
    calculate_optical_flow, 
    predict_class_probas, 
    smooth_probas, 
    threshold_probas, 
    compile_into_events, 
    filter_events, 
    extract_blink_events,
)

from blink_detector.helper import (
    preprocess_frames, 
    get_video_frames, 
    get_timestamps,
    OfParams,
    PPParams,
    create_grid,
    get_classifier,
    show_video,
)

/cluster/anaconda3/envs/tom_py310/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


<font size = 5>**(1) Run blink detection pipeline using an example Neon recording**<br><br>
<font size = 3>In the following, the blink detection algorithm is applied to an
example<br> recording provided with this git repository.

**(1.1) Load example recording**

In [2]:
recording_path = pathlib.Path("/users/tom/experiments/neon_blink_detection/datasets/train_data/padel_tennis_neon_03-2ded8f56")

left_imgs, right_imgs = get_video_frames(recording_path)

timestamps = get_timestamps(recording_path)

left_imgs, right_imgs = preprocess_frames(left_imgs, right_imgs, is_neon=True)

clf = get_classifier(is_neon=True)

**(1.2) Define optical flow and postprocessing parameters** <br>
Take default parameters for both

In [3]:
of_params = OfParams()
pp_params = PPParams()
grid = create_grid(of_params.img_shape, of_params.grid_size)

**(1.3) Predict blink events from video frames**

In [4]:
images_timestamps = zip(zip(left_imgs, right_imgs), timestamps)

x = calculate_optical_flow(images_timestamps, of_params, grid)
x = predict_class_probas(x, clf, of_params)
x = smooth_probas(x, pp_params)
x = threshold_probas(x, pp_params)
x = compile_into_events(x)
x = filter_events(x)

blink_events = list(extract_blink_events(x, pp_params))

<font size = 5><b>Visualize blinks</b><br><br>
<font size=3>In order to visualize the blinks, we will add a white square in the center <br> 
of each frame that is part of a blink event and create an animation from selected frames. 

In [ ]:
from IPython.display import HTML
from blink_detector.helper import show_video

# video length in frames
video_length = 3000

blink_indices = np.zeros_like(timestamps)

for blink_event in blink_events:
    on = np.where(blink_event.start_time == timestamps)[0]
    off = np.where(blink_event.end_time == timestamps)[0]
    blink_indices[int(on):int(off)] = 1

anim = show_video(left_imgs[:video_length, :, :], right_imgs[:video_length, :, :], indices = blink_indices[:video_length])

HTML(anim.to_html5_video())

<font size = 5><b>(2) Blink detection in real time using PL's realtime API</b>

<font size = 3>In the next steps, we will detect blink events in (quasi) real-time using Pupil Lab's Real Time API. <br>
The temporal resolution of the blink detection is restricted by the optical flow parameters (i.e., the number)<br>
of layers and the layer interval, as well as postprocessing parameters (e.g. the minimum blink duration)

**(2.1) Load all relevant modules and functions**

In [10]:
from pupil_labs.realtime_api.simple import discover_one_device
from blink_detector.blink_detector import blink_detection_pipeline
from itertools import tee
import time
import nest_asyncio
import cv2

**(2.2) Set up real time API**

In [ ]:
# needed when running in notebook
nest_asyncio.apply()

# calling the two functions too close together seems to cause a crash, therefore we wait a second
time.sleep(1)

device = discover_one_device()

# check if correct device is connected
print(f"Phone IP address: {device.phone_ip}")
print(f"Phone name: {device.phone_name}")

In [ ]:
# stream and resize frames from realtime API
def video_steam(device):
    while True:
        bgr_pixels, frame_datetime = device.receive_eyes_video_frame()

        left_images = cv2.resize(bgr_pixels[:, :192, 0], (64, 64), interpolation=3)
        right_images = cv2.resize(bgr_pixels[:, 192:, 0], (64, 64), interpolation=3)

        yield left_images, right_images, frame_datetime

In [ ]:
# create three copies of the video stream generator and extract the left and right images and timestamps from each
sl, sr, st = tee(video_steam(device),3)
frames_left = (l for l, r, t in sl)
frames_right = (r for l, r, t in sr)
timestamps = (1e9*t for l, r, t in st)        

# run the blink detection pipeline and obtain blink events in quasi real time
for blinks in blink_detection_pipeline(frames_left, frames_right, timestamps, clf_path=clf_path):
    print(blinks)

    

**Visualize**